In [ ]:
!pip install transformers
!pip install datasets
!pip install accelerate
!pip install transformers[torch]
!pip install evaluate

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [2]:
import pandas as pd
import re
import torch
import string
from datasets import Dataset, DatasetDict
from sklearn.model_selection import train_test_split
import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('/content/drive/MyDrive/faq chatbot/WikiPakistan.csv',header = None,names=['text'])

In [4]:
df = df.dropna(subset=['text'])
df = df[df['text'].str.strip() != '']
df['text'] = df['text'].astype(str)
df['text'] = df['text'].str.lower()
# Remove leading and trailing whitespace
df['text'] = df['text'].str.strip()
# Remove any extra whitespace within the text
df['text'] = df['text'].str.replace(r'\s+', ' ')
df.reset_index(inplace = True)
df.drop('index',axis = 1,inplace = True)

In [5]:
# Create a dictionary containing your data
data = {
    "text": df["text"].tolist()
}

# Convert the dictionary into a Dataset
custom_dataset = Dataset.from_dict(data)

In [6]:
custom_dataset['text'][1]

"pakistan (urdu: پَاکِسْتَان [ˈpaːkɪstaːn]),[d] officially the islamic republic of pakistan (iso: اِسْلامی جَمْہُورِیَہ پَاکِسْتَان, islāmi jamhūriyāh pākistān), is a country in south asia. it is the world's fifth-most populous country, with a population of 241.5 million people, and has the world's largest muslim population as of 2023.[10] islamabad is the nation's capital, while karachi is its largest city and financial centre, followed by lahore, faisalabad, multan, peshawar and hyderabad. pakistan is the 33rd-largest country in the world by area and the second largest in south asia, spanning 881,913 square kilometres (340,509 square miles). it has a 1,046-kilometre (650-mile) coastline along the arabian sea and gulf of oman in the south, and is bordered by india to the east, afghanistan to the west, iran to the southwest, and china to the northeast. it is separated from tajikistan by afghanistan's narrow wakhan corridor in the north, and also shares a maritime border with oman."

In [7]:
# Split the dataset into training and testing sets
dataset = custom_dataset.train_test_split(test_size=0.1, shuffle=True)

In [8]:
dataset.flatten()

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 7098
    })
    test: Dataset({
        features: ['text'],
        num_rows: 789
    })
})

In [9]:
from transformers import AutoTokenizer

# initialize tokenizer
tokenizer = AutoTokenizer.from_pretrained('distilgpt2')
tokenizer.pad_token = tokenizer.eos_token
# # save tokenizer
# tokenizer.save_pretrained('./my_model_directory')

In [10]:
context_length = 128

def tokenizing_text(examples):
    outputs = tokenizer(
        examples['text'],
        padding = 'max_length',
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True
    )

    # Copy input IDs to labels
    outputs["labels"] = outputs["input_ids"].copy()

    return outputs


In [11]:
tokenized_dataset = dataset.map(
    tokenizing_text,
    batched=True,
    num_proc=4,
    remove_columns=dataset["train"].column_names,
)

Map (num_proc=4):   0%|          | 0/7098 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/789 [00:00<?, ? examples/s]

In [12]:
inputs = tokenizer.decode(tokenized_dataset['train']['input_ids'][123])
print(inputs)

his first battlefield experience was with an artillery regiment during the intense fighting for the khemkaran sector in the second kashmir war.[33] he also participated in the lahore and sialkot war zones during the conflict.[19] during the war, musharraf developed a reputation for sticking to his post under shellfire.[15] he received the imtiazi sanad medal for gallantry.[17][20]<|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|><|endoftext|>


In [13]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping', 'labels'],
        num_rows: 9965
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'length', 'overflow_to_sample_mapping', 'labels'],
        num_rows: 1096
    })
})

In [14]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

In [15]:
from transformers import AutoModelForCausalLM, TrainingArguments, Trainer

model = AutoModelForCausalLM.from_pretrained("distilgpt2")

In [16]:
training_args = TrainingArguments(
    output_dir="clm-model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    data_collator=data_collator,
    tokenizer=tokenizer
)

trainer.train()

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,3.724100,3.513039
2,3.513200,3.435144
3,3.429000,3.416431


TrainOutput(global_step=3738, training_loss=3.579831143252157, metrics={'train_runtime': 814.0076, 'train_samples_per_second': 36.726, 'train_steps_per_second': 4.592, 'total_flos': 976433294868480.0, 'train_loss': 3.579831143252157, 'epoch': 3.0})

In [17]:
import math

eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 30.46


In [18]:
trainer.push_to_hub()

pytorch_model.bin:   0%|          | 0.00/328M [00:00<?, ?B/s]

'https://huggingface.co/ammarahad17/clm-model/tree/main/'

In [1]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("ammarahad17/Pakwiki_model")
model = AutoModelForCausalLM.from_pretrained("ammarahad17/Pakwiki_model")

C:\Users\ahada\AppData\Roaming\Python\Python311\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\ahada\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\ahada\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate deve

In [4]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="ammarahad17/Pakwiki_model")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
